In [1]:
import os
import pandas as pd
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import re

In [9]:

sys.path.insert(0, '/directory/tothe/handshakefile/')
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')

mfdfaDataFrames = '/media/ak/T7/August11th2022Experiments/mfdfaDataFrames'  # Input Files
LinearMMDOutputFiles = '/media/ak/T7/August11th2022Experiments/ExperimentOne/TestMMDOutputFiles'  # Location to Save Files
experimentOne = '/media/ak/T7/August11th2022Experiments/ExperimentOne'

In [10]:
def process_symbol(symbol):
    symbol_analyzer = SymbolAnalyzer(symbol=symbol, experimentOne=experimentOne,
                                     bars=['tick', 'calendar', 'dollar', 'volume'])
    symbol_analyzer.check_directory()
    symbol_analyzer.process_bars(max_workers=4)
    
    
def process_symbols(symbols, max_symbol_analyzers):
    """
    Process a list of symbols in parallel.

    :param symbols: list, a list of symbols to process.
    :param max_symbol_analyzers: int, the maximum number of concurrent symbol analyzers.
    """
    with ThreadPoolExecutor(max_workers=max_symbol_analyzers) as executor:
        futures = [executor.submit(process_symbol, symbol) for symbol in symbols]

    for future in futures:
        future.result()

class SymbolAnalyzer:
    def __init__(self, symbol='FB1', experimentOne='', bars=None):
        self.symbol = symbol
        self.symbolPath = os.path.join(experimentOne, str(self.symbol))
        self.resultsMainPath = '/media/ak/T71/August11th2022Experiments/mfdfaDataFrames'
        self.resultsPath = os.path.join(self.resultsMainPath, self.symbol, 'results')
        os.makedirs(self.resultsPath, exist_ok=True)
        self.bars = bars or ['tick']
        self.variables = ['n_F', 'list_H', 'list_H_intercept', 'tau', 'alpha', 'mfSpect']

    def check_directory(self):
        print(self.symbolPath, os.path.isdir(self.symbolPath))

    def get_files(self, bar):
        files = [f for f in os.listdir(self.symbolPath) if str(bar) in f]
        return files

    def process_file(self, file, bar, variable):
        file_loc = os.path.join(self.symbolPath, file)
        variable_array = pd.read_pickle(file_loc)[str(bar)][str(variable)]
        index = file.split('_')[1].split('.')[0]
        #Extract the integer from the column name
#         index = re.findall(r'\d+', file.split('_')[1].split('.')[0])[0]
        
        #result_dict[index] = variable_array
        return index, variable_array

    def process_files_parallel(self, files, bar, max_workers=4):
        for variable in self.variables:
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                results = [executor.submit(self.process_file, file, bar, variable) for file in files]
            result_dict = {r.result()[0]: r.result()[1] for r in results}
            bar_results_path = os.path.join(self.resultsPath, bar)
            os.makedirs(bar_results_path, exist_ok=True)
            save_path = os.path.join(bar_results_path, f"{self.symbol}_{bar}_{variable}.pkl")
            self.save_dataframe_to_pickle(pd.DataFrame(result_dict), save_path)

    def save_dataframe_to_pickle(self, df, save_path, protocol=pickle.HIGHEST_PROTOCOL):
        with open(save_path, 'wb') as f:
            pickle.dump(df, f, protocol=protocol)
        print(f'saving: {save_path}')

    def process_bars(self, max_workers=4):
        for bar in self.bars:
            files = self.get_files(bar)
            self.process_files_parallel(files, bar, max_workers)




In [11]:
if __name__ == '__main__':

    experimentOne = experimentOne
    symbols = ['TY1', 'RX1', 'XM1']
    max_symbol_analyzers = 4

    process_symbols(symbols, max_symbol_analyzers)

PermissionError: [Errno 13] Permission denied: '/media/ak/T71'

In [12]:
file_title =str('/media/ak/T7/August11th2022Experiments/mfdfaDataFrames/RX1/results/volume/RX1_volume_list_H.pkl')
pd.read_pickle(file_title)

,4volume,5volume,20volume,30volume,31volume,0volume,1volume,2volume,3volume,6volume,...,32volume,33volume,34volume,35volume,36volume,37volume,38volume,40volume,42volume,44volume
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,-0.193966,-0.191966,-0.193167,-0.192242,-0.192063,0.279962,0.283290,0.277995,0.315130,0.282745,...,0.273630,0.331860,0.281504,0.319434,-0.192525,0.281958,0.279562,0.279611,0.136913,0.277154
66,-0.201854,-0.199909,-0.201072,-0.200174,-0.200004,0.272230,0.275475,0.270573,0.307616,0.275036,...,0.266214,0.324030,0.273725,0.311746,-0.200444,0.274197,0.271902,0.271961,0.128947,0.269524
67,-0.209316,-0.207423,-0.208549,-0.207678,-0.207516,0.264913,0.268081,0.263542,0.300501,0.267738,...,0.259188,0.316612,0.266365,0.304464,-0.207936,0.266851,0.264650,0.264719,0.121403,0.262300
68,-0.216385,-0.214541,-0.215633,-0.214786,-0.214633,0.257978,0.261075,0.256870,0.293751,0.260818,...,0.252523,0.309573,0.259390,0.297555,-0.215033,0.259888,0.257775,0.257852,0.114248,0.255452


In [13]:
os.listdir(mfdfaDataFrames)

['G_1',
 'YM1',
 'XM1',
 'TU1',
 'TY1',
 'KE1',
 'FB1',
 'JB1',
 'FV1',
 'results',
 'OE1',
 'RX1',
 'DU1',
 'US1']

In [44]:
G_1 = os.path.join(experimentOne, 'G_1')
YM1 = os.path.join(experimentOne, 'YM1')
TY1 = os.path.join(experimentOne, 'TY1')

In [50]:
barChoice = 'calendar'
barChoiceFiles = [f for f in os.listdir(G_1) if str(barChoice) in f]
symbolFilePath = G_1

In [51]:
problems =dict()
ok = dict()
for index, file_name in enumerate(barChoiceFiles):
    try:
        print(pd.read_pickle(os.path.join(symbolFilePath,file_name)))
        ok[index] = os.path.join(symbolFilePath,file_name)
    except ValueError:
        print('problem here:', '------------------')
        print(os.path.join(symbolFilePath,file_name))
        problems[index] = os.path.join(symbolFilePath,file_name)
        
    

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1        0.000025
2       -0.000083
3        0.000141
4       -0.000090
           ...   
48394   -0.000009
48395    0.000012
48396    0.000006
48397    0.000095
48398    0.000005
Name: pct_change_micro_price, Length: 48399, dtype: float64, 'arrival_rates': 0        31.0
1         5.0
2        13.0
3        22.0
4        21.0
         ... 
48394     0.0
48395     0.0
48396     0.0
48397     0.0
48398     0.0
Name: arrival_rates, Length: 48399, dtype: float64, 'gk_vol': 0        0.000241
1        0.000220
2        0.000220
3        0.000224
4        0.000219
           ...   
48394    0.000000
48395    0.000000
48396    0.000000
48397    0.000000
48398    0.000000
Name: GK_vol, Length: 48399, dtype: float64, 'median_traded_volume': 0         5.0
1         8.0
2         9.0
3         5.5
4        10.0
         ... 
48394     2.0
48395     2.0
48396     2.5
48397     1.5
48398     1.0
Name: median_traded_volume, Length: 

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1       -0.000024
2       -0.000085
3        0.000194
4        0.000098
           ...   
44520   -0.000046
44521    0.000038
44522   -0.000026
44523    0.000081
44524    0.000028
Name: pct_change_micro_price, Length: 44525, dtype: float64, 'arrival_rates': 0        24.0
1        18.0
2        20.0
3        15.0
4        13.0
         ... 
44520     0.0
44521     0.0
44522     0.0
44523     0.0
44524     0.0
Name: arrival_rates, Length: 44525, dtype: float64, 'gk_vol': 0        0.000260
1        0.000254
2        0.000251
3        0.000243
4        0.000249
           ...   
44520    0.000000
44521    0.000000
44522    0.000000
44523    0.000000
44524    0.000000
Name: GK_vol, Length: 44525, dtype: float64, 'median_traded_volume': 0         1.5
1         3.0
2         3.0
3         2.5
4         4.0
         ... 
44520     3.0
44521     3.0
44522     2.0
44523    22.0
44524    22.0
Name: median_traded_volume, Length: 

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1        0.000051
2        0.000027
3       -0.000034
4        0.000072
           ...   
43258    0.000011
43259    0.000129
43260   -0.000128
43261    0.000028
43262    0.000070
Name: pct_change_micro_price, Length: 43263, dtype: float64, 'arrival_rates': 0        15.0
1        27.0
2        13.0
3         3.0
4         2.0
         ... 
43258     0.0
43259     0.0
43260     0.0
43261     0.0
43262     0.0
Name: arrival_rates, Length: 43263, dtype: float64, 'gk_vol': 0        0.000247
1        0.000230
2        0.000225
3        0.000216
4        0.000219
           ...   
43258    0.000000
43259    0.000000
43260    0.000000
43261    0.000000
43262    0.000000
Name: GK_vol, Length: 43263, dtype: float64, 'median_traded_volume': 0         5.0
1         3.5
2         8.0
3        10.0
4        10.5
         ... 
43258     2.0
43259     2.0
43260     2.0
43261     1.5
43262     1.0
Name: median_traded_volume, Length: 

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1        0.000019
2       -0.000118
3        0.000021
4        0.000093
           ...   
44244   -0.000114
44245    0.000028
44246    0.000015
44247   -0.000061
44248   -0.000070
Name: pct_change_micro_price, Length: 44249, dtype: float64, 'arrival_rates': 0        11.0
1         1.0
2         4.0
3        31.0
4        18.0
         ... 
44244     0.0
44245     0.0
44246     0.0
44247     0.0
44248     0.0
Name: arrival_rates, Length: 44249, dtype: float64, 'gk_vol': 0        0.000218
1        0.000197
2        0.000197
3        0.000195
4        0.000200
           ...   
44244    0.000000
44245    0.000000
44246    0.000000
44247    0.000000
44248    0.000000
Name: GK_vol, Length: 44249, dtype: float64, 'median_traded_volume': 0        18.0
1        23.0
2        12.5
3         2.0
4         5.0
         ... 
44244     2.0
44245     2.5
44246     1.5
44247     1.0
44248     1.0
Name: median_traded_volume, Length: 

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1        0.000166
2       -0.000011
3       -0.000004
4       -0.000243
           ...   
41458    0.000057
41459   -0.000010
41460   -0.000001
41461   -0.000040
41462    0.000271
Name: pct_change_micro_price, Length: 41463, dtype: float64, 'arrival_rates': 0         6.0
1         5.0
2        18.0
3        15.0
4        30.0
         ... 
41458     0.0
41459     0.0
41460     0.0
41461     0.0
41462     0.0
Name: arrival_rates, Length: 41463, dtype: float64, 'gk_vol': 0        0.000233
1        0.000236
2        0.000232
3        0.000231
4        0.000241
           ...   
41458    0.000000
41459    0.000000
41460    0.000000
41461    0.000000
41462    0.000000
Name: GK_vol, Length: 41463, dtype: float64, 'median_traded_volume': 0        13.0
1        10.5
2         9.5
3        11.0
4         2.5
         ... 
41458     2.5
41459     2.5
41460     1.0
41461     1.5
41462     2.5
Name: median_traded_volume, Length: 

...

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1        0.000080
2       -0.000073
3       -0.000145
4       -0.000034
           ...   
14653   -0.000017
14654   -0.000356
14655   -0.000077
14656   -0.000042
14657   -0.000124
Name: pct_change_micro_price, Length: 14658, dtype: float64, 'arrival_rates': 0        45.0
1        55.0
2        51.0
3        36.0
4        67.0
         ... 
14653     0.0
14654     0.0
14655     0.0
14656     0.0
14657     0.0
Name: arrival_rates, Length: 14658, dtype: float64, 'gk_vol': 0        0.000323
1        0.000340
2        0.000337
3        0.000338
4        0.000336
           ...   
14653    0.000000
14654    0.000000
14655    0.000000
14656    0.000000
14657    0.000000
Name: GK_vol, Length: 14658, dtype: float64, 'median_traded_volume': 0        1.0
1        3.0
2        2.5
3        1.0
4        1.0
        ... 
14653    2.5
14654    1.0
14655    1.0
14656    1.0
14657    3.0
Name: median_traded_volume, Length: 14658, dtyp

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1        0.000051
2        0.000027
3       -0.000034
4        0.000072
           ...   
30901   -0.000024
30902   -0.000003
30903    0.000102
30904    0.000014
30905   -0.000085
Name: pct_change_micro_price, Length: 30906, dtype: float64, 'arrival_rates': 0        15.0
1        27.0
2        13.0
3         3.0
4         2.0
         ... 
30901     0.0
30902     0.0
30903     0.0
30904     0.0
30905     0.0
Name: arrival_rates, Length: 30906, dtype: float64, 'gk_vol': 0        0.000247
1        0.000230
2        0.000225
3        0.000216
4        0.000219
           ...   
30901    0.000000
30902    0.000000
30903    0.000000
30904    0.000000
30905    0.000000
Name: GK_vol, Length: 30906, dtype: float64, 'median_traded_volume': 0         5.0
1         3.5
2         8.0
3        10.0
4        10.5
         ... 
30901     3.0
30902     1.0
30903     3.5
30904     2.5
30905     1.0
Name: median_traded_volume, Length: 

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000
1        0.000025
2       -0.000083
3        0.000141
4       -0.000090
           ...   
30366    0.000035
30367    0.000070
30368   -0.000011
30369    0.000037
30370    0.000062
Name: pct_change_micro_price, Length: 30371, dtype: float64, 'arrival_rates': 0        31.0
1         5.0
2        13.0
3        22.0
4        21.0
         ... 
30366     0.0
30367     0.0
30368     0.0
30369     0.0
30370     0.0
Name: arrival_rates, Length: 30371, dtype: float64, 'gk_vol': 0        0.000241
1        0.000220
2        0.000220
3        0.000224
4        0.000219
           ...   
30366    0.000000
30367    0.000000
30368    0.000000
30369    0.000000
30370    0.000000
Name: GK_vol, Length: 30371, dtype: float64, 'median_traded_volume': 0         5.0
1         8.0
2         9.0
3         5.5
4        10.0
         ... 
30366     3.5
30367     2.0
30368     1.0
30369     4.5
30370    10.0
Name: median_traded_volume, Length: 

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000e+00
1        8.022494e-05
2       -7.293031e-05
3       -1.450654e-04
4       -3.434108e-05
             ...     
14099   -1.078399e-04
14100   -8.297730e-05
14101    7.882612e-05
14102   -6.990773e-08
14103    1.344396e-05
Name: pct_change_micro_price, Length: 14104, dtype: float64, 'arrival_rates': 0        45.0
1        55.0
2        51.0
3        36.0
4        67.0
         ... 
14099     0.0
14100     0.0
14101     0.0
14102     0.0
14103     0.0
Name: arrival_rates, Length: 14104, dtype: float64, 'gk_vol': 0        0.000323
1        0.000340
2        0.000337
3        0.000338
4        0.000336
           ...   
14099    0.000000
14100    0.000000
14101    0.000000
14102    0.000000
14103    0.000000
Name: GK_vol, Length: 14104, dtype: float64, 'median_traded_volume': 0        1.0
1        3.0
2        2.5
3        1.0
4        1.0
        ... 
14099    1.0
14100    1.5
14101    1.0
14102    1.0
14103    1.0
Nam

defaultdict(<class 'dict'>, {'micro_price_change': 0        0.000000e+00
1        1.655560e-04
2       -1.071634e-05
3       -4.202533e-06
4       -2.428109e-04
             ...     
30548    2.107270e-06
30549    1.850024e-08
30550   -2.053785e-06
30551    1.568087e-04
30552   -1.690151e-04
Name: pct_change_micro_price, Length: 30553, dtype: float64, 'arrival_rates': 0         6.0
1         5.0
2        18.0
3        15.0
4        30.0
         ... 
30548     0.0
30549     0.0
30550     0.0
30551     0.0
30552     0.0
Name: arrival_rates, Length: 30553, dtype: float64, 'gk_vol': 0        0.000233
1        0.000236
2        0.000232
3        0.000231
4        0.000241
           ...   
30548    0.000000
30549    0.000000
30550    0.000000
30551    0.000000
30552    0.000000
Name: GK_vol, Length: 30553, dtype: float64, 'median_traded_volume': 0        13.0
1        10.5
2         9.5
3        11.0
4         2.5
         ... 
30548     2.5
30549     2.5
30550     2.0
30551     3.0
30552 

In [64]:
alphas = dict()
taus = dict()
for i in list(ok.keys()):
    alphas[i] = pd.read_pickle(ok[0])[str(barChoice)]['alpha']
    taus[i] =  pd.read_pickle(ok[0])[str(barChoice)]['tau']

In [62]:
pd.DataFrame.from_dict(alphas)

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,...,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514,0.844514
1,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,...,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656,0.840656
2,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,...,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226,0.836226
3,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,...,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081,0.831081
4,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,...,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046,0.825046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,...,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817,0.003817
65,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,...,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792,0.003792
66,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,...,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802,0.003802
67,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,...,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836,0.003836
